## Install sklearn

In [ ]:
!pip install scikit-learn pandas


## Load the dataset

In [9]:
import pandas as pd

df = pd.read_csv("/Users/prabh/Projects/Cistra/dataset/cost-of-living-processed.csv")
df.head()


,City,meal_inexpensive,meal_midrange_2p,mcmeal,beer_domestic,beer_imported_bottle,soda,water,milk,bread,...,cappuccino,rice,tomatoes,bananas,onions,beef,car_price,pre_school,avg_primary_school,median
0,"Saint Petersburg, Russia",7.34,29.35,4.40,2.20,2.20,0.76,0.53,0.98,0.71,...,1.96,0.92,1.91,0.89,0.48,7.18,19305.29,411.83,5388.86,4.40
1,"Istanbul, Turkey",4.58,15.28,3.82,3.06,3.06,0.64,0.24,0.71,0.36,...,1.84,1.30,0.80,1.91,0.62,9.73,20874.72,282.94,6905.43,3.82
2,"Izmir, Turkey",3.06,12.22,3.06,2.29,2.75,0.61,0.22,0.65,0.38,...,1.56,1.31,0.70,1.78,0.58,8.61,20898.83,212.18,4948.41,3.06
3,"Helsinki, Finland",12.00,65.00,8.00,6.50,6.75,2.66,1.89,0.96,2.27,...,3.87,2.13,2.91,1.61,1.25,12.34,24402.77,351.60,1641.00,8.00
4,"Chisinau, Moldova",4.67,20.74,4.15,1.04,1.43,0.64,0.44,0.68,0.33,...,1.25,0.93,1.56,1.37,0.59,5.37,17238.13,210.52,2679.30,3.61


## Show columns 

In [10]:
df.columns

Index(['City', 'meal_inexpensive', 'meal_midrange_2p', 'mcmeal',
       'beer_domestic', 'beer_imported_bottle', 'soda', 'water', 'milk',
       'bread', 'eggs', 'cheese', 'water_large', 'wine',
       'beer_domestic_bottle', 'beer_imported_bottle.1', 'cigarettes',
       'ticket_oneway', 'chicken', 'monthly_pass', 'gasoline',
       'volkswagen_golf', 'apt1_centre', 'apt1_outside', 'apt3_centre',
       'apt3_outside', 'utilities', 'mobile_minute', 'avg_internet',
       'fitness_club', 'tennis_court', 'cinema', 'jeans', 'summer_dress',
       'nike_shoes', 'leather_shoes', 'sqm_buy_centre', 'sqm_buy_outside',
       'salary', 'interest_rate', 'taxi_start', 'taxi_1km', 'taxi_wait',
       'apples', 'oranges', 'potatoes', 'lettuce', 'cappuccino', 'rice',
       'tomatoes', 'bananas', 'onions', 'beef', 'car_price', 'pre_school',
       'avg_primary_school', 'median'],
      dtype='object')

## encode the cities

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['City'] = le.fit_transform(df['City'])


## X and y 

In [15]:
X = df.drop(columns=['median'])
y = df['median']

## 70 - 30 % split for traning and testing

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.30,   # 30% testing
    random_state=42,  # keeps results consistent
    shuffle=True
)

print("Training size:", X_train.shape)
print("Testing size:", X_test.shape)

Training size: (112, 56)
Testing size: (48, 56)


## Ml Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

model.fit(X_train, y_train)

print("Model trained successfully!")


Model trained successfully!


##Evaluate Model

In [22]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

y_pred = model.predict(X_test)

print("R2 Score:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


R2 Score: 0.9063208075680415
RMSE: 0.749767777551575


## Check Score

In [ ]:
from 

## Prediction function

In [19]:
def predict_cost(city_name, data_row):
    # Encode city name
    city_encoded = le.transform([city_name])[0]

    # Override the city value in the row
    data_row['City'] = city_encoded

    # Make sure the order matches X columns
    data_row = data_row[X.columns]

    # Predict
    prediction = model.predict([data_row])[0]
    return prediction


## Test Model

In [21]:
# # Pick one sample from the test set
# sample = X_test.iloc[0]

# # Predict using the trained model
# predicted_cost = model.predict([sample])[0]

# # Actual value from y_test
# actual_cost = y_test.iloc[0]

# print("Predicted Cost:", predicted_cost)
# print("Actual Cost:", actual_cost)
sample = X_test.iloc[[0]]   # ← double brackets keep feature names

predicted_cost = model.predict(sample)[0]
actual_cost = y_test.iloc[0]

print("Predicted:", predicted_cost)
print("Actual:", actual_cost)


Predicted: 3.4810999999999965
Actual: 2.7
